In [1]:
# Your ID : sign in https://gportal.jaxa.jp/gpr/?lang=ja

# Product Name : You can check from https://gportal.jaxa.jp/gpr/assets/mng_upload/GCOM-C/GCOM-C_SHIKISAI_Data_Users_Handbook_jp.pdf
#                page 4-20
        
# Version : '1','2','3'
#             Different versions provide data for different periods
#             check detail from https://shikisai.jaxa.jp/repro_stat_j.html
                
# Date : e.g. '20220322'

# Temporal : daily：'01D' , 8-days ：'08D' , Monthly : '01M'

# Obrit : Ascending：'A' , Descending：'D'
    
# Target_path : e.g. 'xxx/SGLI/'


        

In [1]:
from ftplib import FTP
class SGLI_L2_Downloader:
    
    def __init__ (self, User_ID , ProductName, Ver, Date , Tile , Obrit ,ParaVer, Target_path):
        self.User_ID = User_ID
        self.ProductName = ProductName
        self.Ver = Ver
        self.Date = Date
        self.Tile = Tile
        self.Obrit = Obrit
        self.ParaVer = ParaVer
        self.Target_path = Target_path
        
        
    def path (self):
        return '/standard/GCOM-C/GCOM-C.SGLI/L2.LAND.{}/{}/{}/{}/{}'.format(self.ProductName, self.Ver, self.Date[0:4], self.Date[4:6], self.Date[6:8])

    def filename (self):
        return 'GC1SG1_{}{}01D_T{}_L2SG_{}Q_{}00{}.h5'.format(self.Date, self.Obrit, self.Tile, self.ProductName, self.Ver, self.ParaVer)
    
    def login (self):
        ftp_addr = 'ftp.gportal.jaxa.jp'
        f = FTP(ftp_addr)
        f.login(self.User_ID,'anonymous')
        
        return f
    
    def download (self, f):
        remote_filepath = self.path()
        f.cwd(remote_filepath)
        list=f.nlst()

        bufsize=1024
        data = open ('{}{}'.format(self.Target_path, self.filename()), 'wb')
        filename = 'RETR ' + self.filename()
        f.retrbinary(filename,data.write,bufsize)
        
    def logout (self, f):
        f.quit()
def reporjection_IMAGE(filename,Band,Target_path):
    
    #print(path)
    os.system('/media/liwei/Data/GCOM-C_RSRF/SGLI_geo_map_linux.exe {} -d Image_data/{} -o {} -s 15'.format(filename,Band,Target_path[:-1]))             
def reporjection_QA_05(filename,Band,Target_path):
    
    #print(path)
    os.system('/media/liwei/Data/GCOM-C_RSRF/SGLI_geo_map_linux.exe {} -d Image_data/{} -o {} -s 15'.format(filename,Band,Target_path[:-1]))         

def reporjection_QA_10(filename,Band,Target_path):
    
    #print(path)
    os.system('/media/liwei/Data/GCOM-C_RSRF/SGLI_geo_map_linux.exe {} -d Image_data/{} -o {} -s 15'.format(filename,Band,Target_path[:-1]))         
        
def mkdir(path):
    folder = os.path.exists(path) 
    if not folder:
        os.makedirs(path)
        
    

        
def SGLI_2_AHI_IMAGE(filepath,output_path):
    geotiff_da = xr.open_rasterio(filepath)
    data = geotiff_da.interp(x=lon_y,y=lat_x,method="nearest")
    data = np.where(data != 65535, data ,np.nan)
    data = data * 0.0001
    data = data.reshape(row,col)
    if b == 'Rs_PI02':
        data = block_reduce(data, block_size=(2,2), func=np.nanmean)
        
    SR_file=open(output_path,'wb')
    data.astype('f4').tofile(SR_file)
    SR_file.close()
    return data



def SGLI_2_AHI_QA(filepath,output_path):
    geotiff_da = xr.open_rasterio(filepath)
    data = geotiff_da.interp(x=lon_y,y=lat_x,method="nearest")
    data = np.where(data != 1, data ,np.nan)
    SR_file=open(output_path,'wb')
    data.astype(int).tofile(SR_file)
    SR_file.close()
    return data




In [2]:

def download_tile(date,save_tile,tile):
    tile_file_name = []
    g = SGLI_L2_Downloader(
        'galiwei ' ,
        'RSRF', 
        '3', 
         date , 
         tile, 
        'D' , 
        '1' ,
         save_tile
        )
    f = g.login()
    g.download(f)
    fn = g.filename()
    tile_file_name.append(fn) 
    g.logout(f)
    
    return tile_file_name

In [4]:
# Reflectance


import os
import gdal
import numpy as np
import xarray as xr
from skimage.measure import block_reduce




            


# band = ['Rs_PI01']
band = ['Rs_PI01','Rs_PI02']




res = 0.005


# site_name = 'ROI 1'
# site = [-38.5,177] # ROI 1
# tile = '1231'

# site_name = 'ROI 2'
# site = [-44.5,170.5] # ROI 2
# tile = '1330'


# site_name = 'ROI 1 test2'
# site = [-38.3,177] # ROI 1
# tile = '1231'

# site_name = 'ROI 1 test1'
# site = [-37.9,177.8] # ROI 1
# tile = '1232'

# site_name = 'test 1'
# site = [16.75,96.5] # ROI 1
# tile = '0727'

# site_name = 'ROI 3'
# site = [27,96.5] # ROI 3
# tile = '0626'# ROI 3

# site_name = 'ROI 6'
# site = [46,114]
# tile = '0425'

# site_name = 'ROI 7'
# site = [35.328,99.052]
# tile = '0526'

site_name = 'ROI 8'
site = [42.990,107.043]
tile = '0425'

# site_name = 'ROI 9'
# site = [41.7,104.6]
# tile = '0425'

# site_name = 'ROI 10'
# site = [49.6,119.4]
# tile = '0425'

u_lat,d_lat = site[0]+0.06,site[0]-0.06
l_lon,r_lon = site[1]-0.06,site[1]+0.06

row = round((u_lat - d_lat) / res)
col = round((r_lon - l_lon) / res)

lat_x = np.linspace(u_lat,d_lat + res,row)

# lat_x = np.linspace(d_lat,u_lat - res,row)
lon_y = np.linspace(l_lon,r_lon - res,col)


d_sgli = []
with open("/home/liwei/Himawari_SGLI/{}_2020_PL.txt".format(site_name), "r") as f:
    for line in f.readlines():
        line = line.strip('\n') 
        if line[0:9] == 'SGLI TIME':
            sgli = [line[10:14],line[15:17],line[18:20]]
            
            d_sgli.append("".join(sgli))

with open("/home/liwei/Himawari_SGLI/{}_2021_PL.txt".format(site_name), "r") as f:
    for line in f.readlines():
        line = line.strip('\n') 
        if line[0:9] == 'SGLI TIME':
            sgli = [line[10:14],line[15:17],line[18:20]]
            
            d_sgli.append("".join(sgli))

                                                                                                                                                                                                                                                                        

for date in d_sgli:
    
    print('Start {}'.format(date))
    save_tile = '/media/liwei/Data/GCOM-C_RSRF_Valid/tile/'
    save_Mosaic = '/media/liwei/Data/GCOM-C_RSRF_Valid/Mosaic/'
    save_reporjection = '/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/'
    try :
        tile_file_name = download_tile(date,save_tile,tile)
    except :
        print( "No file {}".format(date))
        pass

    else:

        for b in band:
            if b[0] == 'R':
                print('Start Reporjection {}'.format(b))
                for fn in tile_file_name:
                    reporjection_IMAGE(save_tile+fn,b,save_reporjection)
                print('Reporjection Finished!')

                save_path = '/media/liwei/Data/GCOM-C_RSRF_Valid/{}/{}/'.format(b,date)
                mkdir(save_path)

                file  = save_reporjection + tile_file_name[0][:-3] + '_{}.tif'.format(b)
                data = SGLI_2_AHI_IMAGE(file,save_path+'{}_{}_{}.dat'.format(b,date,site_name))


Start 20200702
Start Reporjection Rs_PI01
========================== Input parameters ==========================
Input file              = /media/liwei/Data/GCOM-C_RSRF_Valid/tile/GC1SG1_20200702D01D_T0425_L2SG_RSRFQ_3001.h5
Projection coordinates  = Geodetic Latitude/Longitude
Pixel spacing           = 15.000000 sec
Resampling method       = BL
Data selection          = Image_data/Rs_PI01
Output directory        = /media/liwei/Data/GCOM-C_RSRF_Valid/reporjection
product_type = L2/tile
tile = 0425
pid = GC1SG1_20200702D01D_T0425_L2SG_RSRFQ_3001
data rank = 2
data size = 1200/1200
data type = unsigned short
INFO: type of attr is int -> cast to unsigned short! function (H5_common_read2.c) line = 272
max_valid_DN = 65534
hemi = 0
x range =   92.6166666667,  124.4583333333
y range =   40.0000000000,   50.0000000000
outsize = 7642, 2400
output = [/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/GC1SG1_20200702D01D_T0425_L2SG_RSRFQ_3001_Rs_PI01.tif]
output = [/media/liwei/Data/GCOM-C_RSRF_Va

/tmp/ipykernel_2428263/3923884669.py:63: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  geotiff_da = xr.open_rasterio(filepath)


========================== Input parameters ==========================
Input file              = /media/liwei/Data/GCOM-C_RSRF_Valid/tile/GC1SG1_20200702D01D_T0425_L2SG_RSRFQ_3001.h5
Projection coordinates  = Geodetic Latitude/Longitude
Pixel spacing           = 15.000000 sec
Resampling method       = BL
Data selection          = Image_data/Rs_PI02
Output directory        = /media/liwei/Data/GCOM-C_RSRF_Valid/reporjection
product_type = L2/tile
tile = 0425
pid = GC1SG1_20200702D01D_T0425_L2SG_RSRFQ_3001
data rank = 2
data size = 1200/1200
data type = unsigned short
INFO: type of attr is int -> cast to unsigned short! function (H5_common_read2.c) line = 272
max_valid_DN = 65534
hemi = 0
x range =   92.6166666667,  124.4583333333
y range =   40.0000000000,   50.0000000000
outsize = 7642, 2400
output = [/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/GC1SG1_20200702D01D_T0425_L2SG_RSRFQ_3001_Rs_PI02.tif]
output = [/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/GC1SG1_20200702D01D_T0425

/tmp/ipykernel_2428263/3923884669.py:63: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  geotiff_da = xr.open_rasterio(filepath)


Start Reporjection Rs_PI01
========================== Input parameters ==========================
Input file              = /media/liwei/Data/GCOM-C_RSRF_Valid/tile/GC1SG1_20200713D01D_T0425_L2SG_RSRFQ_3001.h5
Projection coordinates  = Geodetic Latitude/Longitude
Pixel spacing           = 15.000000 sec
Resampling method       = BL
Data selection          = Image_data/Rs_PI01
Output directory        = /media/liwei/Data/GCOM-C_RSRF_Valid/reporjection
product_type = L2/tile
tile = 0425
pid = GC1SG1_20200713D01D_T0425_L2SG_RSRFQ_3001
data rank = 2
data size = 1200/1200
data type = unsigned short
INFO: type of attr is int -> cast to unsigned short! function (H5_common_read2.c) line = 272
max_valid_DN = 65534
hemi = 0
x range =   91.3750000000,  124.4583333333
y range =   40.0000000000,   50.0000000000
outsize = 7940, 2400
output = [/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/GC1SG1_20200713D01D_T0425_L2SG_RSRFQ_3001_Rs_PI01.tif]
output = [/media/liwei/Data/GCOM-C_RSRF_Valid/reporjectio

...

/tmp/ipykernel_2428263/3923884669.py:63: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  geotiff_da = xr.open_rasterio(filepath)


Start Reporjection Rs_PI01
========================== Input parameters ==========================
Input file              = /media/liwei/Data/GCOM-C_RSRF_Valid/tile/GC1SG1_20210622D01D_T0425_L2SG_RSRFQ_3001.h5
Projection coordinates  = Geodetic Latitude/Longitude
Pixel spacing           = 15.000000 sec
Resampling method       = BL
Data selection          = Image_data/Rs_PI01
Output directory        = /media/liwei/Data/GCOM-C_RSRF_Valid/reporjection
product_type = L2/tile
tile = 0425
pid = GC1SG1_20210622D01D_T0425_L2SG_RSRFQ_3001
data rank = 2
data size = 1200/1200
data type = unsigned short
INFO: type of attr is int -> cast to unsigned short! function (H5_common_read2.c) line = 272
max_valid_DN = 65534
hemi = 0
x range =   92.3208333333,  124.4583333333
y range =   40.0000000000,   50.0000000000
outsize = 7713, 2400
output = [/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/GC1SG1_20210622D01D_T0425_L2SG_RSRFQ_3001_Rs_PI01.tif]
output = [/media/liwei/Data/GCOM-C_RSRF_Valid/reporjectio

/tmp/ipykernel_2428263/3923884669.py:63: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  geotiff_da = xr.open_rasterio(filepath)


Start Reporjection Rs_PI02
========================== Input parameters ==========================
Input file              = /media/liwei/Data/GCOM-C_RSRF_Valid/tile/GC1SG1_20210622D01D_T0425_L2SG_RSRFQ_3001.h5
Projection coordinates  = Geodetic Latitude/Longitude
Pixel spacing           = 15.000000 sec
Resampling method       = BL
Data selection          = Image_data/Rs_PI02
Output directory        = /media/liwei/Data/GCOM-C_RSRF_Valid/reporjection
product_type = L2/tile
tile = 0425
pid = GC1SG1_20210622D01D_T0425_L2SG_RSRFQ_3001
data rank = 2
data size = 1200/1200
data type = unsigned short
INFO: type of attr is int -> cast to unsigned short! function (H5_common_read2.c) line = 272
max_valid_DN = 65534
hemi = 0
x range =   92.3208333333,  124.4583333333
y range =   40.0000000000,   50.0000000000
outsize = 7713, 2400
output = [/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/GC1SG1_20210622D01D_T0425_L2SG_RSRFQ_3001_Rs_PI02.tif]
output = [/media/liwei/Data/GCOM-C_RSRF_Valid/reporjectio

/tmp/ipykernel_2428263/3923884669.py:63: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  geotiff_da = xr.open_rasterio(filepath)


In [4]:
# QA flag

band = ['QA_flag']

res = 0.005
                                                                                                                                                                                                                                                                       
for date in d_sgli:
    
    print('Start {}'.format(date))
    save_tile = '/media/liwei/Data/GCOM-C_RSRF_Valid/tile/'
    save_Mosaic = '/media/liwei/Data/GCOM-C_RSRF_Valid/Mosaic/'
    save_reporjection = '/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/'
    try :
        tile_file_name = download_tile(date,save_tile,tile)
    except :
        print( "No file {}".format(date))
        pass

    else:

        for b in band:
            if b[0] == 'Q':
                print('Start Reporjection {}'.format(b))
                for fn in tile_file_name:
                    reporjection_QA_05(save_tile+fn,b,save_reporjection)
                print('Reporjection Finished!')

                save_path = '/media/liwei/Data/GCOM-C_RSRF_Valid/{}/{}/'.format(b,date)
                mkdir(save_path)

                file  = save_reporjection + tile_file_name[0][:-3] + '_{}.tif'.format(b)
                data = SGLI_2_AHI_QA(file,save_path+'{}_{}_{}_05.dat'.format(b,date,site_name))


Start 20200714
Start Reporjection QA_flag


free(): invalid next size (fast)
Aborted (core dumped)
/tmp/ipykernel_1932310/3923884669.py:79: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  geotiff_da = xr.open_rasterio(filepath)


Reporjection Finished!
Start 20200729
Start Reporjection QA_flag


free(): invalid next size (fast)
Aborted (core dumped)
/tmp/ipykernel_1932310/3923884669.py:79: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  geotiff_da = xr.open_rasterio(filepath)


Reporjection Finished!
Start 20200802
Start Reporjection QA_flag


...

free(): invalid next size (fast)
Aborted (core dumped)
/tmp/ipykernel_1932310/3923884669.py:79: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  geotiff_da = xr.open_rasterio(filepath)


Reporjection Finished!
Start 20210501
Start Reporjection QA_flag


free(): invalid next size (fast)
Aborted (core dumped)
/tmp/ipykernel_1932310/3923884669.py:79: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  geotiff_da = xr.open_rasterio(filepath)


Reporjection Finished!
Start 20210505
Start Reporjection QA_flag
Reporjection Finished!


free(): invalid next size (fast)
Aborted (core dumped)
/tmp/ipykernel_1932310/3923884669.py:79: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  geotiff_da = xr.open_rasterio(filepath)


In [10]:
from ftplib import FTP
class SGLI_L2_Downloader:
    
    def __init__ (self, User_ID , ProductName, Ver, Date , Tile , Obrit ,ParaVer, Target_path):
        self.User_ID = User_ID
        self.ProductName = ProductName
        self.Ver = Ver
        self.Date = Date
        self.Tile = Tile
        self.Obrit = Obrit
        self.ParaVer = ParaVer
        self.Target_path = Target_path
        
        
    def path (self):
        return '/standard/GCOM-C/GCOM-C.SGLI/L2.LAND.{}/{}/{}/{}/{}'.format(self.ProductName, self.Ver, self.Date[0:4], self.Date[4:6], self.Date[6:8])

    def filename (self):
        return 'GC1SG1_{}{}01D_T{}_L2SG_{}Q_{}01{}.h5'.format(self.Date, self.Obrit, self.Tile, self.ProductName, self.Ver, self.ParaVer)
    
    def login (self):
        ftp_addr = 'ftp.gportal.jaxa.jp'
        f = FTP(ftp_addr)
        f.login(self.User_ID,'anonymous')
        
        return f
    
    def download (self, f):
        remote_filepath = self.path()
        f.cwd(remote_filepath)
        list=f.nlst()

        bufsize=1024
        data = open ('{}{}'.format(self.Target_path, self.filename()), 'wb')
        filename = 'RETR ' + self.filename()
        f.retrbinary(filename,data.write,bufsize)
        
    def logout (self, f):
        f.quit()
    
def reporjection_IMAGE(filename,Band,Target_path):
    
    #print(path)
    os.system('/media/liwei/Data/GCOM-C_RSRF/SGLI_geo_map_linux.exe {} -d Image_data/{} -o {} -s 7.5'.format(filename,Band,Target_path[:-1]))         
        
def mkdir(path):
    folder = os.path.exists(path) 
    if not folder:
        os.makedirs(path)
        
    

        
def SGLI_2_AHI_IMAGE(filepath,output_path):
    geotiff_da = xr.open_rasterio(filepath)
    data = geotiff_da.interp(x=lon_y,y=lat_x,method="nearest")
    data = np.where(data != 65535, data ,np.nan)
    data = data.reshape(row,col)

    if b == 'Lt_PI01':
        data = data * 0.00661397 - 66.22
        data = data.reshape(row,col)
    elif b == 'Lt_PI02':
        data = data * 0.00893582 - 89.4667
        data = block_reduce(data, block_size=(2,2), func=np.nanmean)
#     
    SR_file=open(output_path,'wb')
    data.astype('f4').tofile(SR_file)
    SR_file.close()
    return data


    


In [11]:

def download_tile(date,save_tile,tile):
    tile_file_name = []
    g = SGLI_L2_Downloader(
        'galiwei ' ,
        'LTOA', 
        '2', 
         date , 
         tile, 
        'D' , 
        '1' ,
         save_tile
        )
    f = g.login()
    g.download(f)
    fn = g.filename()
    tile_file_name.append(fn) 
    g.logout(f)
    
    return tile_file_name

In [12]:
# Radiance


import os
import gdal
import numpy as np
import xarray as xr
from skimage.measure import block_reduce



# band = ['Rs_PI01']
# band = ['Lt_PI02']
band = ['Lt_PI01','Lt_PI02']

                                                                                                                                                                                                                                                               

for date in d_sgli:
    
    print('Start {}'.format(date))
    save_tile = '/media/liwei/Data/GCOM-C_RSRF_Valid/tile/'
    save_Mosaic = '/media/liwei/Data/GCOM-C_RSRF_Valid/Mosaic/'
    save_reporjection = '/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/'
    try :
        tile_file_name = download_tile(date,save_tile,tile)
    except :
        print( "No file {}".format(date))
        pass

    else:

        for b in band:
            if b[0] == 'L':
                print('Start Reporjection {}'.format(b))
                for fn in tile_file_name:
                    reporjection_IMAGE(save_tile+fn,b,save_reporjection)
                print('Reporjection Finished!')

                save_path = '/media/liwei/Data/GCOM-C_RSRF_Valid/{}/{}/'.format(b,date)
                mkdir(save_path)

                file  = save_reporjection + tile_file_name[0][:-3] + '_{}.tif'.format(b)
                data = SGLI_2_AHI_IMAGE(file,save_path+'{}_{}_{}_rad.dat'.format(b,date,site_name))


Start 20200714
Start Reporjection Lt_PI01
========================== Input parameters ==========================
Input file              = /media/liwei/Data/GCOM-C_RSRF_Valid/tile/GC1SG1_20200714D01D_T0425_L2SG_LTOAQ_2011.h5
Projection coordinates  = Geodetic Latitude/Longitude
Pixel spacing           = 7.500000 sec
Resampling method       = BL
Data selection          = Image_data/Lt_PI01
Output directory        = /media/liwei/Data/GCOM-C_RSRF_Valid/reporjection
product_type = L2/tile
tile = 0425
pid = GC1SG1_20200714D01D_T0425_L2SG_LTOAQ_2011
data rank = 2
data size = 1200/1200
data type = unsigned short
max_valid_DN = 65534
hemi = 0
x range =   96.6854166667,  124.4583333333
y range =   40.0000000000,   50.0000000000
outsize = 13331, 4800
output = [/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/GC1SG1_20200714D01D_T0425_L2SG_LTOAQ_2011_Lt_PI01.tif]
output = [/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/GC1SG1_20200714D01D_T0425_L2SG_LTOAQ_2011_Lt_PI01.xml]
done...
Reporjection 

/tmp/ipykernel_1776687/1007119598.py:55: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  geotiff_da = xr.open_rasterio(filepath)


========================== Input parameters ==========================
Input file              = /media/liwei/Data/GCOM-C_RSRF_Valid/tile/GC1SG1_20200714D01D_T0425_L2SG_LTOAQ_2011.h5
Projection coordinates  = Geodetic Latitude/Longitude
Pixel spacing           = 7.500000 sec
Resampling method       = BL
Data selection          = Image_data/Lt_PI02
Output directory        = /media/liwei/Data/GCOM-C_RSRF_Valid/reporjection
product_type = L2/tile
tile = 0425
pid = GC1SG1_20200714D01D_T0425_L2SG_LTOAQ_2011
data rank = 2
data size = 1200/1200
data type = unsigned short
max_valid_DN = 65534
hemi = 0
x range =   96.7083333333,  124.4583333333
y range =   40.0000000000,   50.0000000000
outsize = 13320, 4800
output = [/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/GC1SG1_20200714D01D_T0425_L2SG_LTOAQ_2011_Lt_PI02.tif]
output = [/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/GC1SG1_20200714D01D_T0425_L2SG_LTOAQ_2011_Lt_PI02.xml]
done...
Reporjection Finished!
Start 20200729


/tmp/ipykernel_1776687/1007119598.py:55: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  geotiff_da = xr.open_rasterio(filepath)


Start Reporjection Lt_PI01
========================== Input parameters ==========================
Input file              = /media/liwei/Data/GCOM-C_RSRF_Valid/tile/GC1SG1_20200729D01D_T0425_L2SG_LTOAQ_2011.h5
Projection coordinates  = Geodetic Latitude/Longitude
Pixel spacing           = 7.500000 sec
Resampling method       = BL
Data selection          = Image_data/Lt_PI01
Output directory        = /media/liwei/Data/GCOM-C_RSRF_Valid/reporjection
product_type = L2/tile
tile = 0425
pid = GC1SG1_20200729D01D_T0425_L2SG_LTOAQ_2011
data rank = 2
data size = 1200/1200
data type = unsigned short
max_valid_DN = 65534
hemi = 0
x range =   95.9687500000,  124.4583333333
y range =   40.0000000000,   50.0000000000
outsize = 13675, 4800
output = [/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/GC1SG1_20200729D01D_T0425_L2SG_LTOAQ_2011_Lt_PI01.tif]
output = [/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/GC1SG1_20200729D01D_T0425_L2SG_LTOAQ_2011_Lt_PI01.xml]
done...
Reporjection Finished!


...

Start Reporjection Lt_PI01
========================== Input parameters ==========================
Input file              = /media/liwei/Data/GCOM-C_RSRF_Valid/tile/GC1SG1_20210505D01D_T0425_L2SG_LTOAQ_2011.h5
Projection coordinates  = Geodetic Latitude/Longitude
Pixel spacing           = 7.500000 sec
Resampling method       = BL
Data selection          = Image_data/Lt_PI01
Output directory        = /media/liwei/Data/GCOM-C_RSRF_Valid/reporjection
product_type = L2/tile
tile = 0425
pid = GC1SG1_20210505D01D_T0425_L2SG_LTOAQ_2011
data rank = 2
data size = 1200/1200
data type = unsigned short
max_valid_DN = 65534
hemi = 0
x range =   99.3833333333,  122.3062500000
y range =   40.0000000000,   50.0000000000
outsize = 11003, 4800
output = [/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/GC1SG1_20210505D01D_T0425_L2SG_LTOAQ_2011_Lt_PI01.tif]
output = [/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/GC1SG1_20210505D01D_T0425_L2SG_LTOAQ_2011_Lt_PI01.xml]
done...
Reporjection Finished!
Start

/tmp/ipykernel_1776687/1007119598.py:55: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  geotiff_da = xr.open_rasterio(filepath)


========================== Input parameters ==========================
Input file              = /media/liwei/Data/GCOM-C_RSRF_Valid/tile/GC1SG1_20210505D01D_T0425_L2SG_LTOAQ_2011.h5
Projection coordinates  = Geodetic Latitude/Longitude
Pixel spacing           = 7.500000 sec
Resampling method       = BL
Data selection          = Image_data/Lt_PI02
Output directory        = /media/liwei/Data/GCOM-C_RSRF_Valid/reporjection
product_type = L2/tile
tile = 0425
pid = GC1SG1_20210505D01D_T0425_L2SG_LTOAQ_2011
data rank = 2
data size = 1200/1200
data type = unsigned short
max_valid_DN = 65534
hemi = 0
x range =   99.4062500000,  122.3062500000
y range =   40.0000000000,   50.0000000000
outsize = 10992, 4800
output = [/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/GC1SG1_20210505D01D_T0425_L2SG_LTOAQ_2011_Lt_PI02.tif]
output = [/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/GC1SG1_20210505D01D_T0425_L2SG_LTOAQ_2011_Lt_PI02.xml]
done...
Reporjection Finished!
Start 20210604


/tmp/ipykernel_1776687/1007119598.py:55: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  geotiff_da = xr.open_rasterio(filepath)


No file 20210604


In [5]:
from ftplib import FTP
class SGLI_L2_Downloader:
    
    def __init__ (self, User_ID , ProductName, Ver, Date , Tile , Obrit ,ParaVer, Target_path):
        self.User_ID = User_ID
        self.ProductName = ProductName
        self.Ver = Ver
        self.Date = Date
        self.Tile = Tile
        self.Obrit = Obrit
        self.ParaVer = ParaVer
        self.Target_path = Target_path
        
        
    def path (self):
        return '/standard/GCOM-C/GCOM-C.SGLI/L2.LAND.{}/{}/{}/{}/{}'.format(self.ProductName, self.Ver, self.Date[0:4], self.Date[4:6], self.Date[6:8])

    def filename (self):
        return 'GC1SG1_{}{}01D_T{}_L2SG_{}Q_{}01{}.h5'.format(self.Date, self.Obrit, self.Tile, self.ProductName, self.Ver, self.ParaVer)
    
    def login (self):
        ftp_addr = 'ftp.gportal.jaxa.jp'
        f = FTP(ftp_addr)
        f.login(self.User_ID,'anonymous')
        
        return f
    
    def download (self, f):
        remote_filepath = self.path()
        f.cwd(remote_filepath)
        list=f.nlst()

        bufsize=1024
        data = open ('{}{}'.format(self.Target_path, self.filename()), 'wb')
        filename = 'RETR ' + self.filename()
        f.retrbinary(filename,data.write,bufsize)
        
    def logout (self, f):
        f.quit()
    
def reporjection_IMAGE(filename,Band,Target_path):
    
    #print(path)
    os.system('/media/liwei/Data/GCOM-C_RSRF/SGLI_geo_map_linux.exe {} -d Image_data/{} -o {} -s 7.5'.format(filename,Band,Target_path[:-1]))         
        
def mkdir(path):
    folder = os.path.exists(path) 
    if not folder:
        os.makedirs(path)
        
    

        
def SGLI_2_AHI_IMAGE(filepath,output_path):
    geotiff_da = xr.open_rasterio(filepath)
    data = geotiff_da.interp(x=lon_y,y=lat_x,method="nearest")
    data = np.where(data != 65535, data ,np.nan)
    data = data.reshape(row,col)

    if b == 'Lt_PI01':
        data = data * 0.0000133603 - 0.133765
        data = data.reshape(row,col)
    elif b == 'Lt_PI02':
        data = data * 0.0000283651 - 0.283995
        data = block_reduce(data, block_size=(2,2), func=np.nanmean)
#     
    SR_file=open(output_path,'wb')
    data.astype('f4').tofile(SR_file)
    SR_file.close()
    return data


    


In [6]:

def download_tile(date,save_tile,tile):
    tile_file_name = []
    g = SGLI_L2_Downloader(
        'galiwei ' ,
        'LTOA', 
        '2', 
         date , 
         tile, 
        'D' , 
        '1' ,
         save_tile
        )
    f = g.login()
    g.download(f)
    fn = g.filename()
    tile_file_name.append(fn) 
    g.logout(f)
    
    return tile_file_name

In [7]:
# Ref


import os
import gdal
import numpy as np
import xarray as xr
from skimage.measure import block_reduce


band = ['Lt_PI01','Lt_PI02']


res = 0.005

                                                                                                                                                                                                                                                                      

for date in d_sgli:
    
    print('Start {}'.format(date))
    save_tile = '/media/liwei/Data/GCOM-C_RSRF_Valid/tile/'
    save_Mosaic = '/media/liwei/Data/GCOM-C_RSRF_Valid/Mosaic/'
    save_reporjection = '/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/'
    try :
        tile_file_name = download_tile(date,save_tile,tile)
    except :
        print( "No file {}".format(date))
        pass

    else:

        for b in band:
            if b[0] == 'L':
                print('Start Reporjection {}'.format(b))
                for fn in tile_file_name:
                    reporjection_IMAGE(save_tile+fn,b,save_reporjection)
                print('Reporjection Finished!')

                save_path = '/media/liwei/Data/GCOM-C_RSRF_Valid/{}/{}/'.format(b,date)
                mkdir(save_path)

                file  = save_reporjection + tile_file_name[0][:-3] + '_{}.tif'.format(b)
                data = SGLI_2_AHI_IMAGE(file,save_path+'{}_{}_{}_toa.dat'.format(b,date,site_name))


Start 20200702
Start Reporjection Lt_PI01
========================== Input parameters ==========================
Input file              = /media/liwei/Data/GCOM-C_RSRF_Valid/tile/GC1SG1_20200702D01D_T0425_L2SG_LTOAQ_2011.h5
Projection coordinates  = Geodetic Latitude/Longitude
Pixel spacing           = 7.500000 sec
Resampling method       = BL
Data selection          = Image_data/Lt_PI01
Output directory        = /media/liwei/Data/GCOM-C_RSRF_Valid/reporjection
product_type = L2/tile
tile = 0425
pid = GC1SG1_20200702D01D_T0425_L2SG_LTOAQ_2011
data rank = 2
data size = 1200/1200
data type = unsigned short
max_valid_DN = 65534
hemi = 0
x range =   92.1937500000,  124.4583333333
y range =   40.0000000000,   50.0000000000
outsize = 15487, 4800
output = [/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/GC1SG1_20200702D01D_T0425_L2SG_LTOAQ_2011_Lt_PI01.tif]
output = [/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/GC1SG1_20200702D01D_T0425_L2SG_LTOAQ_2011_Lt_PI01.xml]
done...
Reporjection 

/tmp/ipykernel_2428263/2503893380.py:55: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  geotiff_da = xr.open_rasterio(filepath)


========================== Input parameters ==========================
Input file              = /media/liwei/Data/GCOM-C_RSRF_Valid/tile/GC1SG1_20200702D01D_T0425_L2SG_LTOAQ_2011.h5
Projection coordinates  = Geodetic Latitude/Longitude
Pixel spacing           = 7.500000 sec
Resampling method       = BL
Data selection          = Image_data/Lt_PI02
Output directory        = /media/liwei/Data/GCOM-C_RSRF_Valid/reporjection
product_type = L2/tile
tile = 0425
pid = GC1SG1_20200702D01D_T0425_L2SG_LTOAQ_2011
data rank = 2
data size = 1200/1200
data type = unsigned short
max_valid_DN = 65534
hemi = 0
x range =   92.2145833333,  124.4583333333
y range =   40.0000000000,   50.0000000000
outsize = 15477, 4800
output = [/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/GC1SG1_20200702D01D_T0425_L2SG_LTOAQ_2011_Lt_PI02.tif]
output = [/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/GC1SG1_20200702D01D_T0425_L2SG_LTOAQ_2011_Lt_PI02.xml]
done...
Reporjection Finished!
Start 20200713


/tmp/ipykernel_2428263/2503893380.py:55: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  geotiff_da = xr.open_rasterio(filepath)


Start Reporjection Lt_PI01
========================== Input parameters ==========================
Input file              = /media/liwei/Data/GCOM-C_RSRF_Valid/tile/GC1SG1_20200713D01D_T0425_L2SG_LTOAQ_2011.h5
Projection coordinates  = Geodetic Latitude/Longitude
Pixel spacing           = 7.500000 sec
Resampling method       = BL
Data selection          = Image_data/Lt_PI01
Output directory        = /media/liwei/Data/GCOM-C_RSRF_Valid/reporjection
product_type = L2/tile
tile = 0425
pid = GC1SG1_20200713D01D_T0425_L2SG_LTOAQ_2011
data rank = 2
data size = 1200/1200
data type = unsigned short
max_valid_DN = 65534
hemi = 0
x range =   91.3770833333,  124.4583333333
y range =   40.0000000000,   50.0000000000
outsize = 15879, 4800
output = [/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/GC1SG1_20200713D01D_T0425_L2SG_LTOAQ_2011_Lt_PI01.tif]
output = [/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/GC1SG1_20200713D01D_T0425_L2SG_LTOAQ_2011_Lt_PI01.xml]
done...
Reporjection Finished!
Start

...

Start Reporjection Lt_PI01
========================== Input parameters ==========================
Input file              = /media/liwei/Data/GCOM-C_RSRF_Valid/tile/GC1SG1_20210519D01D_T0425_L2SG_LTOAQ_2011.h5
Projection coordinates  = Geodetic Latitude/Longitude
Pixel spacing           = 7.500000 sec
Resampling method       = BL
Data selection          = Image_data/Lt_PI01
Output directory        = /media/liwei/Data/GCOM-C_RSRF_Valid/reporjection
product_type = L2/tile
tile = 0425
pid = GC1SG1_20210519D01D_T0425_L2SG_LTOAQ_2011
data rank = 2
data size = 1200/1200
data type = unsigned short
max_valid_DN = 65534
hemi = 0
x range =   91.9437500000,  124.4583333333
y range =   40.0000000000,   50.0000000000
outsize = 15607, 4800
output = [/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/GC1SG1_20210519D01D_T0425_L2SG_LTOAQ_2011_Lt_PI01.tif]
output = [/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/GC1SG1_20210519D01D_T0425_L2SG_LTOAQ_2011_Lt_PI01.xml]
done...
Reporjection Finished!
Start

/tmp/ipykernel_2428263/2503893380.py:55: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  geotiff_da = xr.open_rasterio(filepath)


========================== Input parameters ==========================
Input file              = /media/liwei/Data/GCOM-C_RSRF_Valid/tile/GC1SG1_20210519D01D_T0425_L2SG_LTOAQ_2011.h5
Projection coordinates  = Geodetic Latitude/Longitude
Pixel spacing           = 7.500000 sec
Resampling method       = BL
Data selection          = Image_data/Lt_PI02
Output directory        = /media/liwei/Data/GCOM-C_RSRF_Valid/reporjection
product_type = L2/tile
tile = 0425
pid = GC1SG1_20210519D01D_T0425_L2SG_LTOAQ_2011
data rank = 2
data size = 1200/1200
data type = unsigned short
max_valid_DN = 65534
hemi = 0
x range =   91.9541666667,  124.4583333333
y range =   40.0000000000,   50.0000000000
outsize = 15602, 4800
output = [/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/GC1SG1_20210519D01D_T0425_L2SG_LTOAQ_2011_Lt_PI02.tif]
output = [/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/GC1SG1_20210519D01D_T0425_L2SG_LTOAQ_2011_Lt_PI02.xml]
done...
Reporjection Finished!
Start 20210603


/tmp/ipykernel_2428263/2503893380.py:55: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  geotiff_da = xr.open_rasterio(filepath)


No file 20210603
Start 20210607
No file 20210607
Start 20210618
No file 20210618
Start 20210622
No file 20210622
